In [9]:
import tensorflow as tf
from mtcnn import MTCNN
from pathlib import Path
import pandas as pd
import glob
import cv2
import csv
import os
import ast
import pydot
import pydotplus
import graphviz
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import time
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import tensorflow_addons as tfa
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback

C:\Users\michael\miniconda3\envs\torch\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\michael\miniconda3\envs\torch\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.

In [7]:
# ===================== MULTITASK MODEL SETUP =====================

base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

existing_output = base_model.output

spatial_attention = tf.keras.layers.Conv2D(2, (1, 1), activation='sigmoid', padding='same')(existing_output)

dropout_layer1 = tf.keras.layers.Dropout(rate=0.2, name="dropout_layer1")(spatial_attention)

spatial_attention = tf.keras.layers.Softmax()(spatial_attention)
output_tensor = tf.keras.layers.Multiply()([spatial_attention])

# Add Global Average Pooling layer
output_tensor = tf.keras.layers.GlobalAveragePooling2D()(output_tensor)

# Add output layer with two classes and softmax activation
prediction = tf.keras.layers.Dense(1, activation='sigmoid', name='drowsiness_output')(output_tensor)

# Create the new model with the modified top layers
drowsiness_model = Model(inputs=base_model.input, outputs=prediction)

drowsiness_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [10]:
drowsiness_model.compile(
    optimizer=Adam(learning_rate=1e-6),
    loss={
        'drowsiness_output': 'binary_crossentropy'
    },
    metrics={
        'drowsiness_output': [
                             tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                             tf.keras.metrics.Precision(name='precision'),
                             tf.keras.metrics.Recall(name='recall'),
                             tfa.metrics.F1Score(num_classes=1, threshold=0.5)]
    }
)

drowsiness_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [11]:
# ===================== ORIGINAL DATA GEN CLASS =====================

class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df, X_col, y_col,
                 batch_size,
                 input_size=(224, 224, 3),
                 shuffle=True,
                 random_seed=None):  # Add a new parameter for random seed
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.random_seed = random_seed  # Store the random seed
        
        self.n = len(self.df)
#         self.n_coords = 2  # Assuming landmark coordinates are 2-dimensional
        self.n_drowsiness = 1  # Assuming a single illuminance value
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1, random_state=self.random_seed).reset_index(drop=True)  # Use the random seed
    
    def __get_input(self, path, target_size):
    
        image = tf.keras.preprocessing.image.load_img(path)
        image_arr = tf.keras.preprocessing.image.img_to_array(image)

        image_arr = tf.image.resize(image_arr, (target_size[0], target_size[1])).numpy()

        return image_arr / 255.
    
    def __get_output(self, label, output_type):
        # Assuming output_type is 'coordinates', 'illuminance', or 'adjusted_image_path'
#         if output_type == 'coordinates':
#             # Assuming label is a string containing a dictionary-like structure
#             # Safely evaluate the string as a literal dictionary using ast.literal_eval
#             coordinates_dict = ast.literal_eval(label)
            
#             # Extract x and y coordinates for each landmark
#             landmarks = ['left_eye', 'right_eye', 'nose', 'mouth_left', 'mouth_right']
#             coordinates_list = [coordinates_dict[landmark] for landmark in landmarks]
            
#             # Flatten the list and convert to numpy array
#             coordinates_array = np.array([coord for landmark_coords in coordinates_list for coord in landmark_coords])
            
# #             print("Shape of landmarks_array:", coordinates_array.shape)
            
#             # If there are exactly 10 values, return the array, otherwise raise an error
#             if len(coordinates_array) == 10:
#                 return coordinates_array
#             else:
#                 raise ValueError("Expected 10 coordinates, but found {}".format(len(coordinates_array)))
#         elif output_type == 'illuminance':
#             # Convert the illuminance value to a float
#             return float(label)
#         elif output_type == 'adjusted_image_path':
#             # Assuming label is the path to the adjusted image
#             return self.__get_input(label, self.input_size)
        if output_type == 'drowsiness':
            # Convert the illuminance value to a float
            return int(label)
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        path_batch = batches[self.X_col['path']]
        
#         coords_batch = batches[self.y_col['coordinates']]
        illuminance_batch = batches[self.y_col['drowsiness']]
#         adjusted_image_path_batch = batches[self.y_col['adjusted_image_path']]

        X_batch = np.asarray([self.__get_input(x, self.input_size) for x in path_batch])

#         y0_batch = np.asarray([self.__get_output(y, 'coordinates') for y in coords_batch])
        y0_batch = np.asarray([self.__get_output(y, 'drowsiness') for y in illuminance_batch])
#         y2_batch = np.asarray([self.__get_output(y, 'adjusted_image_path') for y in adjusted_image_path_batch])

        return X_batch, [y0_batch]
    
    def __getitem__(self, index):
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)

        # Print a few examples of the data
#         print("Sample X:", tf.shape(X[0]))  # Print the first example in the batch
#         print("Sample y[0] (landmarks):", tf.shape(y[0][0]))  # Print the first example in the landmarks output
#         print("Sample y[1] (illum):", tf.shape(y[1][0]))  # Print the first example in the illum output
#         print("Sample y[2] (adjusted_image_path):", tf.shape(y[2][0]))  # Print the first example in the adjusted_image_path output

        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [12]:
# ===================== DATA GEN SETUP (DROWSINESS TASK) =====================

test_df = pd.read_csv("./data/training.csv") # path to test_data csv
test_df["Filename"] = "./data/Training/" + test_df["Filename"]

def getDrowsiness(filename):
    index = filename.rfind('.')
    return filename[index - 1] if index >= 1 else None

test_df["Drowsiness"] = test_df["Filename"].apply(getDrowsiness).astype(int)


eval_df = pd.read_csv("./data/evaluation.csv") # path to test_data csv
eval_df["Filename"] = "./data/Evaluation/Evaluation/" + eval_df["Filename"]

def getDrowsiness(filename):
    index = filename.rfind('.')
    return filename[index - 1] if index >= 1 else None

eval_df["Drowsiness"] = eval_df["Filename"].apply(getDrowsiness).astype(int)


test_gen = CustomDataGen(df=test_df, X_col={'path': 'Filename'}, y_col={'drowsiness': 'Drowsiness'}, batch_size=32, input_size=(224, 224, 3), random_seed=438)
eval_gen = CustomDataGen(df=eval_df, X_col={'path': 'Filename'}, y_col={'drowsiness': 'Drowsiness'}, batch_size=32, input_size=(224, 224, 3), random_seed=438)

In [14]:
class DynamicLearningRateScheduler(Callback):
    def __init__(self, monitor_metric='val_loss', patience=3, factor=0.5, min_lr=1e-6):
        super(DynamicLearningRateScheduler, self).__init__()
        self.monitor_metric = monitor_metric
        self.patience = patience
        self.factor = factor
        self.min_lr = min_lr
        self.wait = 0
        self.best_metric = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        current_metric = logs.get(self.monitor_metric)

        if current_metric is None:
            raise ValueError(f"Metric {self.monitor_metric} not found in training logs.")

        if current_metric < self.best_metric:
            self.best_metric = current_metric
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                new_lr = max(self.model.optimizer.lr * self.factor, self.min_lr)
                self.model.optimizer.lr = new_lr
                print(f"\nLearning rate reduced to {new_lr}.")
                self.wait = 0

dynamicLearningCallback = DynamicLearningRateScheduler(monitor_metric='val_loss', patience=3, factor=0.5, min_lr=1e-8)

In [15]:
evaluation_result = drowsiness_model.fit(test_gen, epochs=10, validation_data=eval_gen, callbacks=[dynamicLearningCallback])

Epoch 1/10
 35/940 [>.............................] - ETA: 15:08 - loss: 0.7019 - accuracy: 0.4527 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.0000e+00

KeyboardInterrupt: 